## Baseline
- https://dacon.io/competitions/official/235736/codeshare/2641?page=1&dtype=recent

In [1]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [2]:
train=pd.read_csv('../data/elec/train.csv',encoding='cp949')
test=pd.read_csv('../data/elec/test.csv', encoding='cp949')
submission=pd.read_csv('../data/elec/sample_submission.csv')

In [3]:
train.shape

(122400, 10)

In [4]:
train.head()

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0


In [5]:
train.tail()

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0
122399,60,2020-08-24 23,3204.576,27.1,2.6,75.0,0.0,0.0,1.0,1.0


In [6]:
test.shape

(10080, 9)

In [7]:
test.head()

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']
    

In [9]:
for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

In [10]:
#시간 변수와 요일 변수를 추가해봅니다.
def time(x):
    return int(x[-2:])
train['time']=train['date_time'].apply(lambda x: time(x))
test['time']=test['date_time'].apply(lambda x: time(x))

def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

test 결측치 보간해줍니다.
test 데이터의 변수는 예보 데이터이며, 예보 데이터는 train 데이터의 기간에 생성된 것이기에 활용 가능합니다.

In [11]:
test.head()

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time,weekday
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,0.0,0.0,0,1
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2,1
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,0.0,0.0,3,1
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,0.0,0.0,4,1


In [12]:
test = test.interpolate(method = 'values')

In [13]:
test.head()

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time,weekday
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.0,0.0,0.0,0,1
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.0,0.0,0.0,1,1
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.0,0.0,0.0,2,1
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.0,0.0,0.0,3,1
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.0,0.0,0.0,4,1


### 모델링

In [14]:
train_x=train.drop('전력사용량(kWh)', axis=1)
train_y=train[['전력사용량(kWh)']]

In [15]:
train_x.drop('date_time', axis=1, inplace=True)
test.drop('date_time', axis=1, inplace=True)

In [16]:
cross=KFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in cross.split(train_x, train_y):
    folds.append((train_idx, valid_idx))

In [17]:
folds

[(array([     1,      2,      4, ..., 122394, 122395, 122397]),
  array([     0,      3,     23, ..., 122396, 122398, 122399])),
 (array([     0,      1,      2, ..., 122397, 122398, 122399]),
  array([     8,     12,     17, ..., 122381, 122387, 122389])),
 (array([     0,      2,      3, ..., 122397, 122398, 122399]),
  array([     1,      4,      6, ..., 122379, 122383, 122394])),
 (array([     0,      1,      2, ..., 122396, 122398, 122399]),
  array([     9,     10,     14, ..., 122388, 122390, 122397])),
 (array([     0,      1,      3, ..., 122397, 122398, 122399]),
  array([     2,      5,     15, ..., 122391, 122393, 122395]))]

In [18]:
models={}
for fold in range(5):
    print(f'===================={fold+1}=======================')
    train_idx, valid_idx=folds[fold]
    X_train=train_x.iloc[train_idx, :]
    y_train=train_y.iloc[train_idx, :]
    X_valid=train_x.iloc[valid_idx, :]
    y_valid=train_y.iloc[valid_idx, :]
    
    model=LGBMRegressor(n_estimators=100)
    model.fit(X_train, y_train, 
              eval_set=[(X_train, y_train), (X_valid, y_valid)], 
              )
    models[fold]=model
    
    print(f'================================================\n\n')

====================1=======================
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007916 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 773
[LightGBM] [Info] Number of data points in the train set: 97920, number of used features: 10
[LightGBM] [Info] Start training from score 2327.154754


====================2=======================
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002131 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 774
[LightGBM] [Info] Number of data points in the train set: 97920, number of used features: 10
[LightGBM] [Info] Start training from score 2324.635440


====================3=======================
[LightGBM] [Info] Auto-choosing row-wise mult

In [19]:
models

{0: LGBMRegressor(),
 1: LGBMRegressor(),
 2: LGBMRegressor(),
 3: LGBMRegressor(),
 4: LGBMRegressor()}

In [20]:
submission

,num_date_time,answer
0,1 2020-08-25 00,0
1,1 2020-08-25 01,0
2,1 2020-08-25 02,0
3,1 2020-08-25 03,0
4,1 2020-08-25 04,0
...,...,...
10075,60 2020-08-31 19,0
10076,60 2020-08-31 20,0
10077,60 2020-08-31 21,0
10078,60 2020-08-31 22,0


In [21]:
for i in range(5):
    submission['answer'] += models[i].predict(test)/5

In [22]:
submission

,num_date_time,answer
0,1 2020-08-25 00,8498.811895
1,1 2020-08-25 01,8472.288652
2,1 2020-08-25 02,8452.945720
3,1 2020-08-25 03,8475.304267
4,1 2020-08-25 04,8481.468380
...,...,...
10075,60 2020-08-31 19,3522.427819
10076,60 2020-08-31 20,3105.800661
10077,60 2020-08-31 21,2987.995728
10078,60 2020-08-31 22,2948.583029
